In [296]:
#===============================================================================
# class Map
#===============================================================================

class Map:
    def __init__(self,cluster=None):
        self.cluster = cluster

    def permanence(self,p):    # encode permanence
        """
        self.permanence(p): convert permanence to symbolic string
        >>> o = Map()
        >>> o.permanence(0.52)
        'B'
        >>> o.permanence([-1,0,0.01,0.49,0.5,0.99,1,2])
        '<0yaAY1>'
        """
        def upper(x):
            return chr(int(65+(x-0.5)*100//2))
        def lower(x):
            return chr(int(65+32+(0.5-x)*100//2))

        if isinstance(p,list):
            s = ''
            for k in range(len(p)):
                s += self.permanence(p[k])
            return s

        if p < 0:
            return '<'
        elif p == 0:
            return '0'
        elif p == 1:
            return '1'
        elif p > 1:
            return '>'
        elif p < 0.5:
            return lower(p)
        elif p >= 0.5:
            return upper(p)
        else:
            return '?'

    def symbol(self,x):
        """
        self.symbol(x): convert index to symbol or vice versa
        >>> o = Map()
        >>> o.symbol(11)
        'B'
        >>> o.symbol([0,1,10,11,35,36,37,61,62])
        '01ABZabz52'
        """
        def symb(x):
            if x < 10:
                return chr(48+x)
            if x < 36:
                return chr(55+x)
            elif x < 62:
                return chr(61+x)
            else:
                return str(x)

        if isinstance(x,int):
            return symb(x)
        elif isinstance(x,list):
            s = ''
            for k in range(len(x)):
                s += self.symbol(x[k])
            return s

    def bar(self,n):                   # bar string of length n
            str = ''
            for k in range(n): str += '-'
            return str

    def head(self,n,c):                # draw a box head with label
        str = ''
        for k in range(n): 
            if k == 1:
                str += '('
            elif k == 2:
                str += c
            elif k == 3:
                str += ')'
            else:
                str += '-'
        return str

    def headline(self,n,s):
        line = '+'
        for j in range(n):
            #line += title(s,23) + '+'
            line += self.bar(s) + '+'
        return line
                           
    def Pmap(self):
            
        def title(n,x):
            return '-%03g-' % x
            
        def weights(cells,i,j,d):
            m,n,dd,s = cells.shape
            W = cells.P[i][j][d]
            str = ''
            for k in range(s):
               str += self.permanence(W[k])
            return str

        cells = self.cluster
        m,n,d,s = cells.shape
        head = self.headline(n,s)
        str = ''
        for i in range(m):
            print(head)
            for mu in range(d):
                line = '|'
                for j in range(n):
                    line += weights(cells,i,j,mu) + '|'
                print(line)
        print(head)

    def Kmap(self):
            
        def title(n,x):
            return '-%03g-' % x
            
        def indices(cells,i,j,d):
            m,n,dd,s = cells.shape
            K = cells.K[i][j][d]
            str = ''
            for k in range(s):
               str += self.symbol(K[k])
            return str

        cells = self.cluster
        m,n,d,s = cells.shape
        head = self.headline(n,s)
        str = ''
        for i in range(m):
            print(head)
            for mu in range(d):
                line = '|'
                for j in range(n):
                    line += indices(cells,i,j,mu) + '|'
                print(line)
        print(head)


In [297]:
o=Map()
print(o.symbol(26))
str(67)
o.symbol([0,1,25,26,27,51,52])
o.bar(5)

Q


'-----'

In [298]:
"""
class Cluster(): parameter definition for neurotron clusters
clu = Cluster(m,n,d,s)
"""

from numpy import array

#=========================================================================
# class Cluster
#=========================================================================

class Cluster:
    def __init__(self,m=4,n=10,d=2,s=5,f=10,g=3):
        def _zeros(d,s):
            return [[0 for j in range(s)] for i in range(d)]
        def _tensor(m,n,d,s):
            return [[_zeros(d,s) for j in range(n)] for i in range(m)]
            
        self.shape = (m,n,d,s)
        self.P = _tensor(m,n,d,s)     # prediction permanences
        self.K = _tensor(m,n,d,s)     # prediction indices
        self.F = _tensor(m,n,1,f)     # excitation weights
        self.G = _tensor(m,n,1,g)     # collaboration weights
        
        self.U = array(_zeros(m,n))
        self.Q = array(_zeros(m,n))
        self.D = array(_zeros(m,n))
        self.B = array(_zeros(m,n))
        self.X = array(_zeros(m,n))
        self.Y = array(_zeros(m,n))
        self.S = array(_zeros(m,n))
        self.L = array(_zeros(m,n))

    def kappa(self,i,j=None):
        """
        self.kappa():  convert matrix indices to linear index or vice versa
        >>> o = Cluster(m=4,n=10)
        >>> k = o.kappa(i:=1,j:=3)  # k = i+j*m 
        13
        >>> ij = o.kappa(k:=13)    # i = k%m, j = k//m
        (1,3)
        """

        m,n,d,s = self.shape
        if j is None:
            k = i
            return (k%m,k//m)
        else:
            return i + j*m

    def setP(self,i,j,P):
        assert isinstance(P,list) and isinstance(P[0],list) 
        self.P[i][j] = P.copy()


In [299]:
cells = Cluster(4,9,2,5)
cells.setP(0,0,[[.5,.3,.2,.2,.1],[0,0,1,0,1]])
Map(cells).Pmap()
Map(cells).Kmap()

+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|Akppu|00000|00000|00000|00000|00000|00000|00000|00000|
|00101|00000|00000|00000|00000|00000|00000|00000|00000|
+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|00000|00000|00000|00000|00000|00000|00000|00000|00000|
|00000|00000|00000|00000|00000|00000|00000|00000|00000|
+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|00000|00000|00000|00000|00000|00000|00000|00000|00000|
|00000|00000|00000|00000|00000|00000|00000|00000|00000|
+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|00000|00000|00000|00000|00000|00000|00000|00000|00000|
|00000|00000|00000|00000|00000|00000|00000|00000|00000|
+-----+-----+-----+-----+-----+-----+-----+-----+-----+
+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|00000|00000|00000|00000|00000|00000|00000|00000|00000|
|00000|00000|00000|00000|00000|00000|00000|00000|00000|
+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|00000|00000|00000|00000|00000|00000|00000|00000

In [294]:
chr(48)

'0'